<a href="https://colab.research.google.com/github/huzaifa-44/Ai-image-generating-application/blob/main/AI_interview_question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio transformers torch accelerate gTTS SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2240, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1747, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [2]:
def generate_questions(role):
    prompt = f"Generate 5 technical interview questions for the job role: {role}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [3]:
def evaluate_answers(role, answers):
    prompt = (
        f"You are an expert interviewer.\n"
        f"Job Role: {role}\n"
        f"Candidate's Answers:\n{answers}\n\n"
        f"Give feedback on correctness, clarity, and technical depth."
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=250)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [4]:
log_history = []

def save_to_log(role, answers):
    from datetime import datetime
    entry = {
        "role": role,
        "answers": answers,
        "time": datetime.now().strftime("%Y-%m-%d %H:%M")
    }
    log_history.append(entry)
    return "✅ Answers saved successfully."

In [5]:
import os
from gtts import gTTS

def full_agent(role, user_answers, mic_audio):
    if not user_answers.strip() and mic_audio is None:
        # Only role input → Generate questions
        questions = generate_questions(role)
        return questions, "", None

    # Convert mic audio to text
    if mic_audio:
        import speech_recognition as sr
        recognizer = sr.Recognizer()
        with sr.AudioFile(mic_audio) as source:
            audio = recognizer.record(source)
            try:
                user_answers = recognizer.recognize_google(audio)
            except:
                user_answers = "[❌ Mic input could not be transcribed]"

    # Evaluate
    feedback = evaluate_answers(role, user_answers)
    save_to_log(role, user_answers)

    # Text-to-Speech
    tts = gTTS(text=feedback)
    tts.save("feedback.mp3")

    return "", feedback, "feedback.mp3"

In [6]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🧠 AI Interview Agent\nGet smart questions and AI-based feedback with voice or text.")

    with gr.Row():
        role_input = gr.Textbox(label="🎯 Job Role (e.g. React Developer at Google)")
        generate_btn = gr.Button("🧠 Generate Questions")

    questions_box = gr.Textbox(label="📋 Interview Questions", lines=5)

    with gr.Row():
        answer_input = gr.Textbox(label="📝 Your Answer", lines=5)
        mic_input = gr.Audio(label="🎤 Mic Input (upload or record)", type="filepath")

    with gr.Row():
        evaluate_btn = gr.Button("📊 Get Feedback")

    feedback_box = gr.Textbox(label="🧠 AI Feedback", lines=6)
    audio_feedback = gr.Audio(label="🔊 Feedback Audio")

    generate_btn.click(
        fn=full_agent,
        inputs=[role_input, answer_input, mic_input],
        outputs=[questions_box, feedback_box, audio_feedback]
    )

    evaluate_btn.click(
        fn=full_agent,
        inputs=[role_input, answer_input, mic_input],
        outputs=[questions_box, feedback_box, audio_feedback]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50206d4a4e2ea2534c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
